# Geometric Algebra Intro

## Motivation

Problems: 
* rotations = confusing
  * quaternions are 4D hyperspheres? wtf? where does the math come from?
  * rotations in 2D, 3D, 4D have completely different implementations
* handling of infinity in projective geometry = confusing  
  * Lots of if statements, special cases
* cross products = confusing  
  * Right hand rule?
* lie groups = confusing   

What we get:
* Naturally "typed" points, vectors, lines, planes
* Intuitive projections, intersections, metrics, norms on all types
* Exception/conditional free code
* Rotations and translations use the same formulas, on all types
* Formulas generalize to any number of dimensions
* Derive complex numbers, quaternions, dual quaternions, lie algebras

Used in more complex math. Quantum physics. Spacetime. Conformal geometry. Also much easier to explain to a child.

 is coordinate-free,  
• has a uniform representation for points, lines, and planes,  
• can calculate “parallel-safe” meet and join of these geometric entities,  
• provides compact expressions for all classical euclidean formulas and  
constructions, including distances and angles, perpendiculars and parallels,  
orthogonal projections, and other metric operations,  
• has a single, geometrically intuitive form for euclidean motions, one  
with a single representation for operators and operands,  
• provides automatic differentiation of functions of one or several variables,  
• provides a compact, efficient model for kinematics and rigid body mechanics,  
• lends itself to efficient, practical implementation, and  

## Basis Symbols

We're going to start with Euclidean space where we define symbols $\{e_1, ..., e_N\}$ represent the bases of $\mathbb{R}^N$. Representing a vector is just like the familiar decomposition into unit basis vectors:

$v = e_1 + 2 e_2 + 3 e_3 \Rightarrow$ analogous to the vector (1, 2, 3)

However, we're going to think of the bases as scalar algebraic symbols, since treating them as unit vectors is a cyclic definition of vector when starting from first principles. Note that two elements of this algebra can be added together as expected:

$(e_1 + 3e_2) + (2 e_1 - e_2)$
$= 3 e_1 + 2 e_2$

What's more interesting is that geometric algebra is built around multiplication of elements. This is in fact the basis of geometric algebra, and the operation is called the *geometric product* even though it's just standard multiplication:

$(e_1 + 3e_2) * (2 e_1 - e_2)$  
$= 2 e_1^2 - e_1 e_2 + 6 e_2 e_1 - 3 e_2^2$

We don't yet know what this result means, but let's look at simpler products before interpreting it.

## Metric Space

First, what should the square of a basis be? If we think of $e_1$ as a unit vector along the X axis, then it's pretty intuitive to consider $e_1^2 = 1$, as equivalent to the squared length. This allows us reason about metric distances like the length of a vector, which is useful in Euclidean space.

More generally, geometric symbols square to one of the scalars $\{1, 0, -1\}$. There are theorems that relate the squared signature of the bases to the type of space it is useful for representing (projective, hyperbolic, elliptical, Minkowski). For Euclidean space in $\mathbb{R}^N$, we use $N$ bases that square to 1. Let's define this as our first rule:

1) $\boxed{e_i^2 = 1}$

If we then take the square of a non-unit vector, we see an interesting property:

$(e_1 + 2 e_2)^2$  
$= e_1^2 + 2 e_1 e_2 + 2 e_2 e_1 + 4 e_2^2$  
$= 1 + 2 e_1 e_2 + 2 e_2 e_1 + 4$  
$= 5 + 2 e_1 e_2 + 2 e_2 e_1$  


If we say we want the geometric square to represent the squared norm of the vector ($1^2 + 2^2 = 5$), we see the only way this will happen is if $2 e_1 e_2$ and $2 e_2 e_1$ cancel out:

$5 + 2 e_1 e_2 + 2 e_2 e_1 = 5$  
$ e_2 e_1 = -e_1 e_2$

Thus, the desire for multiplying a vector with itself to result in a scalar that represents its squared length necessarily produces our second rule, that the product of two different bases is anti-commutative:

2) $\boxed{e_i e_j = -e_j e_i}$

There are many ways to derive these two rules, but they are the all we need to explore a ton of geometric algebra. Let's go back to our earlier product of two vectors:

$(e_1 + 3e_2) * (2 e_1 - e_2)$  
$= 2 e_1^2 - e_1 e_2 + 6 e_2 e_1 - 3 e_2^2$  

We can now apply the two rules to simplify:

$= 2 - e_1 e_2 + 6 e_2 e_1 - 3 \Rightarrow$  (apply Rule 1, $e_i^2 = 1$)  
$= 2 - e_1 e_2 + 6 (- e_1 e_2) - 3 \Rightarrow$  (apply Rule 2, $e_i e_j = -e_j e_i$)  
$= -1 - 7 e_1 e_2$

## Inner and outer products

The product of any one-dimensional vectors always produces a zero-dimensional quantity which is a contraction, plus a two-dimensional quantity which is an expansion. This is a simple result of the simplification rules, since the squared terms become scalars and the cross terms remain. The best way to think of this compound result is that in one expression it tells us two different things about the relationship between the vectors.

We call the zero-dimensional quantity the *inner product* and the two-dimensional quantity the *outer product*. We can decompose these two parts by writing the inner as ($a \cdot b$) and the outer as ($a \wedge b$):

$a * b = a b = a \cdot b + a \wedge b$

**NOTE**: The formula above applies only to the product of two one-dimensional vectors. The inner product is just the name given to the lowest-dimensional result of the product of any two things, and the outer product is the highest-dimensional result. In general you can get other in-between terms, so they should really be thought of as parts of the geometric product rather than first-class operations themselves.

So from our original product we can write:

$(e_1 + 3e_2) \cdot (2 e_1 - e_2) = -1 \Rightarrow$ (inner product)  
$(e_1 + 3 e_2) \wedge (2 e_1 - e_2) = -7 e_1 e_2 \Rightarrow$ (outer product)

If you look at the product and how the squared terms contract, it's not hard to see that the inner product between two Euclidean vectors is identical to the dot product. However, the inner product applies to any two geometric quantities (planes, lines, points, transformations) and not just vectors. It also applies to non-Euclidean spaces.

What does $e_1 e_2$ actually represent? Fairly intuitively, it represents a unit area in the XY plane. The outer product of two one-dimensional vectors, which can be thought of as *oriented lengths*, produces a two-dimensional *oriented area*. This reinforces why the outer product is anti-commutative - the area $e_1 e_2$ is oriented in the opposite direction of $e_2 e_1$.

<img style="float: left; width: 200px;" src="https://www.researchgate.net/profile/A_Arsenovic/publication/318475103/figure/fig6/AS:547679924756491@1507588477780/Geometric-algebra-for-three-dimensional-space-illustrating-the-vector-and-bivector-basis.png"/>

From this you can see close similarities to the cross product - the magnitude is the signed area of the parallelogram spanned by the two vectors, and the direction is the normal of the $e_1 e_2$ plane. But again, the outer product applies in every dimension and between any two quantities. It has to, since it's literally just the highest-dimensional result of multiplying two elements of the algebra. This property is a lot more intuitive than any derivation of the cross product - it's just part of multiplication.

Extending it one step further, the outer product of three basis vectors produces an oriented volume:

<img style="float: left; width: 250px;" src="https://i.stack.imgur.com/UNRIn.png"/>

Note some basic properties that come out of the two rules:

For bases $e_1, e_2$:  
$e_1 \cdot e_1 = 1$  
$e_1 \cdot e_2 = 0$  
$e_1 \wedge e_1 = 0$  
$e_1 \wedge e_2 = e_1 e_2 = -e_2 e_1 = -e_2 \wedge e_1$  

For vectors $a, b$:  
$a b = a \cdot b + a \wedge b$  
$a \cdot b = (a b + b a) / 2 \Rightarrow$  (inner product is symmetric)  
$a \wedge b = (a b - b a) / 2 \Rightarrow$  (outer product is anti-symmetric)  


## Multivectors

The expression $-1 - 7 e_1 e_2$ is not a one-dimensional vector. A general element of this algebra is called a *multivector*, which is the sum of $k$-vectors from dimension zero to $N$. In general we define a *$k$-vector* as a sum of terms which all have $k$ basis elements. Here are some examples:
 * $42$ is a 0-vector  
 * $3 e_1 + 2 e_2$ is a 1-vector  
 * $3 e_1 e_2 + 5 e_1 e_3$ is a 2-vector.
 * $2 e_1 e_2 e_3$ is a 3-vector.  
 * $42 + 3 e_1$ is a multivector which is the sum of a 0-vector and a 1-vector.  
 * $e_2 - e_1 e_2$ is a multivector which is the sum of a 1-vector and a 2-vector.

The typical names given to these quantities are:
 * 0-vector -> scalar
 * 1-vector -> vector (oriented length)
 * 2-vector -> bivector (oriented area)
 * 3-vector -> trivector (oriented volume)

In $\mathbb{R}^3$, note how there.

The two-dimensional component of the result is called the *outer product* of the vectors, whereas the zero-dimensional (scalar) component is called the *inner product* of the vectors. Note that we are just attributing names to parts of the product (also called *geometric product*). For two vectors ONLY, the product is always the sum of the inner ($a \cdot b$) and outer ($a \wedge b$) products:

$a * b = a b = a \cdot b + a \wedge b$

This is not true for the product of things other than vectors, because we can get in-between terms. It's best to think of the geometric product with the two simplification rules as the first-class entity, and the inner/outer products as selecting the lowest and highest dimensional component of the result.

The inner/outer products are similar in concept to standard dot and cross products, but generalize to any number of dimensions and to points/lines/planes/transforms and not just vectors. They will be very useful operations for geometry. Note some basic properties that come out of our rules:

What does $e_1 e_2$ actually represent? Well, if a basis can be considered an *oriented length*, then the product of two bases intuitively produces an *oriented area*. In the image below, consider the oriented area as the area of the parallelogram spanned by the two vectors, in the plane of the two vectors:

![bivector](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRkct-_-DdOgfKzNk-8m10tz6vPumZN-2D9qxPosZUZFAm1GxCOKA&s)

In [13]:
# Define R3 bases with sympy, which supports non-commutative symbols.
import sympy as sm
e1, e2, e3 = sm.symbols('e1 e2 e3', commutative=False)

Define an expression that is analogous to the vector (1, 2, -3):

In [14]:
vec = e1 + 2 * e2 - 3 * e3
vec

e1 + 2*e2 - 3*e3

We've been taught we can't "multiply" two vectors, but there's nothing stopping an open-minded person from doing so, and is in fact the basis of geometric algebra.

In [19]:
v1 = e1 + 2 * e2
v2 = 3 * e1 - e2
v1 * v2

(e1 + 2*e2)*(3*e1 - e2)

In [20]:
sm.expand(v1 * v2)

-e1*e2 + 3*e1**2 + 6*e2*e1 - 2*e2**2

What does this product mean? Well, if a vector can be considered an *oriented length*, then the product of two vectors intuitively produces an *oriented area*. In the image below, consider the oriented area as the area of the parallelogram spanned by the two vectors, in the plane of the two vectors:

![bivector](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRkct-_-DdOgfKzNk-8m10tz6vPumZN-2D9qxPosZUZFAm1GxCOKA&s)

Notice how if $a$ and $b$ are reversed, then the orientations are backwards. This leads us to add our first algebraic rule, that the product of two basis vectors are anti-commutative:

1) $\boxed{e_i e_j = -e_j e_i}$

From our product above, we can then simplify

$-e_1 e_2 + 6 e_2 e_1 = -e_1 e_2 + 6 (-e_1 e_2) = -7 e_1 e_2$

Since $e_1$ and $e_2$ are unit length, that $e_1 e_2$ is a unit in the XY plane, pointed to the +Z axis. Then $-7 e_1 e_2$ is an area of 7 pointed to the -Z axis.

Now, intuitively squared terms like $3 e_1^2$ produce a zero area, since it spans a degenerate parallelogram. It is true that we get zero area, however to enable doing metric operations in euclidian space, we define the square of bases to be the scalar 1 instead of 0:

2) $\boxed{e_i^2 = 1}$

For different types of spaces (projective, hyperbolic, elliptical, Minkowski), different bases square to one of $\{1, 0, -1\}$. There is a theorem that relates the algebras defined by these rules to the spaces it can represent. But starting with Euclidian space through the origin, we square to 1. Ultimately we pick these rules to be the simplest set that are useful for building up higher level concepts.

Note that Rule 1 is a similar concept to the cross-product, and Rule 2 is similar to the dot product. In general, a vector is a one-dimensional quantity, and multiplication with another vector produces a two-dimensional quantity (oriented area) from the cross terms, plus a zero-dimensional (scalar) quantity from the squared terms:

$(e_1 + 2 e_2) (3 e_1 - e_2)$  
$= -e_1 e_2 + 3e_1^2 + 6 e_2 e_1 - 2 e_2^2$  
$= -e_1 e_2 + 3(1) + 6(-e_1 e_2) - 2 (1)$  
$= -7 e_1 e_2 + 1$

The two-dimensional component of the result is called the *outer product* of the vectors, whereas the zero-dimensional (scalar) component is called the *inner product* of the vectors. Note that we are just attributing names to parts of the product (also called *geometric product*). For two vectors ONLY, the product is always the sum of the inner ($a \cdot b$) and outer ($a \wedge b$) products:

$a * b = a b = a \cdot b + a \wedge b$

This is not true for the product of things other than vectors, because we can get in-between terms. It's best to think of the geometric product with the two simplification rules as the first-class entity, and the inner/outer products as selecting the lowest and highest dimensional component of the result.

The inner/outer products are similar in concept to standard dot and cross products, but generalize to any number of dimensions and to points/lines/planes/transforms and not just vectors. They will be very useful operations for geometry. Note some basic properties that come out of our rules:

For bases:  
$e_1 \cdot e_1 = 1$  
$e_1 \cdot e_2 = 0$  
$e_1 \wedge e_1 = 0$  
$e_1 \wedge e_2 = e_1 e_2 = -e_2 e_1 = -e_2 \wedge e_1$  

For $a, b$ as vectors only:  
$a b = a \cdot b + a \wedge b$  
$a \cdot b = (a b + b a) / 2$  (inner product is symmetric)  
$a \wedge b = (a b - b a) / 2$  (outer product is anti-symmetric)  


## Simplification
Let's create a function that automatically applies our two algebraic rules.

In [45]:
def simp(a):
    """
    Apply simplifications to bases until no changes occur.
    """
    subs = {
        # Rule 1) Squared bases contract to a scalar (1 for Euclidian space)
        e1**2: 1,
        e2**2: 1,
        e3**2: 1,
        # Rule 2) Product of two different are anti-commutative
        e2*e1: -e1*e2,
        e3*e1: -e1*e3,
        e3*e2: -e2*e3
    }
    while True:
        b = sm.expand(a).subs(subs)
        if b == a:
            return b
        a = b

See that this behaves as expected on some simple examples:

In [46]:
simp(e1 * e1)  # Contracts

1

In [47]:
simp(e1 * e2)  # Nothing happens

e1*e2

In [48]:
simp(e2 * e1)  # Anti-commutes to our chosen canonical order

-e1*e2

In [49]:
simp(e2 * e1 * e2)  # Anti-commutes then contracts

-e1

Now let's simplify our vector-vector product from before:

In [50]:
expr = sm.expand((e1 + 2 * e2) * (3 * e1 - e2))
expr

-e1*e2 + 3*e1**2 + 6*e2*e1 - 2*e2**2

In [51]:
simp(expr)

1 - 7*e1*e2

## Multivectors

The expression $1 - 7 e_1 e_2$ is not a one-dimensional vector. A general element of this algebra is called a *multivector*, which is the sum of $k$-vectors from dimension zero to $N$. In general we define a *$k$-vector* as a sum of terms which all have $k$ basis elements. Here are some examples:
 * $42$ is a 0-vector  
 * $3 e_1 + 2 e_2$ is a 1-vector  
 * $3 e_1 e_2 + 5 e_1 e_3$ is a 2-vector.
 * $2 e_1 e_2 e_3$ is a 3-vector.  
 * $42 + 3 e_1$ is a multivector which is the sum of a 0-vector and a 1-vector.  
 * $e_2 - e_1 e_2$ is a multivector which is the sum of a 1-vector and a 2-vector.
 
In $\mathbb{R}^N$, note how we can only have a  Note how the simplifcation rules will cancel any higher order terms out:

In [333]:
simp(e1 * e2 * e3 * e2 * e2)

e1*e2*e3

In [334]:
simp(e1 * e2 * e3 * e2 * e2 * e1 * e2)

-e3

The highest order element we can have for $\mathbb{R}^3$ is the $3$-vector $e_1 e_2 e_3$. The general form of a multivector in this space is:

$c_1 + c_2 e_1 + c_3 e_2 + c_4 e_3 + c_5 e_1 e_2 + c_6 e_1 e_3 + c_7 e_2 e_3 + c_8 e_1 e_2 e_3$

In $\mathbb{R}^2$ we have only two bases and one bivector:

$c_1 + c_2 e_1 + c_3 e_2 + c_4 e_1 e_2$

Mathematically, this algebra is a ring - a set that defines addition and multiplication. It is also a closed group - any element of the algebra will have the above form for some coefficients $c$. It is commutative under addition but non-commutative under multiplication. It is always associative.

So far we've defined the geometric algebra for $\mathbb{R}^N$ but not interpreted or used it for geometry.

## Meaning
If $e_1$ and $e_2$ are unit vectors, what does the bivector $e_1 e_2$ represent? Intuitively, it represents an *oriented area*, in the same way that a vector represents an *oriented length*. It can be thought of as the plane defined by the two vectors, with an area spanned by their parallelogram:

<img style="float: left; width: 200px;" src="https://www.researchgate.net/profile/A_Arsenovic/publication/318475103/figure/fig6/AS:547679924756491@1507588477780/Geometric-algebra-for-three-dimensional-space-illustrating-the-vector-and-bivector-basis.png"/>

Similarly, a trivector represents an oriented volume. In $\mathbb{R}^3$ it's more like a signed volume since there is only one trivector $e_{123}$.

<img style="float: left; width: 250px;" src="https://i.stack.imgur.com/UNRIn.png"/>

The following is the general case for the product of two vectors in $\mathbb{R}^2$:

In [335]:
a, b, c, d = sm.symbols('a b c d')
simp((a * e1 + b * e2) * (c * e1 + d * e2))

a*c + a*d*e1*e2 - b*c*e1*e2 + b*d

Note how the product is a multivector that is the sum of a scalar $ac + bd$ and a bivector $(ad - bc) e_1 e_2$. How do we interpret this? Well, they are both interesting quantities but we generally treat them separately. The 

grade operation

$a = (ac + bd) + (ad - bc) e_1 e_2$  
$\langle a \rangle_0 = ac + bd$  
$\langle a \rangle_1 = 0$  
$\langle a \rangle_2 = (ad - bc) e_1 e_2$  
$\langle a \rangle_3 = 0$  

inner product, outer product

In [336]:
simp((e1 + 2*e2 + 3*e3)**2)

14

## Homogenous (projective) space
cayley tables

## Motors

exponential

## Lie Algebra

foiefwe